# ParaView Volume Rendering with parallel tasks on the local node

In [ ]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule

we first launch a parallel MPI pvserver on the node, using a terminal
The jupyter client will connect to its server on the same node

#
srun  -n 8 pvserver

In [ ]:
Connect("localhost")

In [ ]:
#gs = GetSettingsProxy("RenderViewSettings")
#gs.SetPropertyWithName("DisableIceT", 0)

rank = vtkProcessModule.GetProcessModule().GetPartitionId()
nbprocs = servermanager.ActiveConnection.GetNumberOfDataPartitions()
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

In [ ]:
Version = str(GetParaViewVersion())
if rank == 0:
    print("ParaView Version ", Version)
    print("rank=", rank, "/", nbprocs)
    print("Vendor:   %s" % info.GetVendor())
    print("Version:  %s" % info.GetVersion())
    print("Renderer: %s" % info.GetRenderer())

In [ ]:
reader = ImageReader(FileNames=['/scratch/snx3000/jfavre/NVIDIA/50.density-log_1024x1024x2048_float.raw'])
reader.DataScalarType = 'float'
reader.DataByteOrder = 'LittleEndian'
reader.DataExtent = [0, 2047, 0, 1023, 0, 1023]

In [ ]:
renderView = GetActiveView()
rep = Show(reader, GetActiveView())
rep.Representation = 'Outline'
rep.ColorArrayName = ['POINTS', 'ImageFile']

ImageFileLUT = GetColorTransferFunction('ImageFile')
ImageFileLUT.RGBPoints = [1.206409215927124, 0.231373, 0.298039, 0.752941, 5.729784369468689, 0.865003, 0.865003, 0.865003, 10.253159523010254, 0.705882, 0.0156863,
 0.14902]
ImageFileLUT.ScalarRangeInitialized = 1.0

# get opacity transfer function/opacity map for 'MetaImage'
ImageFilePWF = GetOpacityTransferFunction('ImageFile')
ImageFilePWF.Points = [1.206409215927124, 0.0, 0.5, 0.0, 10.253159523010254, 1.0, 0.5, 0.0]
ImageFilePWF.ScalarRangeInitialized = 1

# Here I recommend to wait until the cell has really finished to run.
# recall that we are running in parallel on the node and some extra sync does not hurt.

In [ ]:
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(GetActiveView())
w = display(pvdisp)

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
SaveImage("/users/jfavre/foo2.png")

In [ ]:
# let us see what are all the view options?
rep.Representation.Available

In [ ]:
rep.Representation = 'Volume'
rep.LookupTable = ImageFileLUT
rep.OpacityArray = ['POINTS', 'ImageFile']
rep.OpacityTransferFunction = 'PiecewiseFunction'
rep.ScalarOpacityUnitDistance = 4
rep.ScalarOpacityFunction = ImageFilePWF

In [ ]:
ResetCamera()

We can cleanly close the connection with the following command

In [ ]:
Disconnect()

In [ ]:
rep.Representation = 'Surface'
ColorBy(rep, 'vtkProcessId')

In [ ]:
processIdLUT = GetColorTransferFunction('vtkProcessId')
processIdLUT.AnnotationsInitialized = 1
processIdLUT.InterpretValuesAsCategories = 1

# we take colors from the pre-defined "KAAMS" found in
# ParaViewCore/ServerManager/Rendering/ColorMaps.json
IndexedColors = [
    1.0, 1.0, 1.0,
    1.0, 0.0, 0.0,
    0.0, 1.0, 0.0,
    0.0, 0.0, 1.0,
    1.0, 1.0, 0.0,
    1.0, 0.0, 1.0,
    0.0, 1.0, 1.0,
    0.63, 0.63, 1.0,
    0.67, 0.5, 0.33,
    1.0, 0.5, 0.75,
    0.53, 0.35, 0.7,
    1.0, 0.75, 0.5
]

a = []
for i in range(nbprocs):
    a.extend((str(i), str(i)))

processIdLUT.Annotations = a
processIdLUT.IndexedColors = IndexedColors